<a href="https://colab.research.google.com/github/Trickshotblaster/ASLClassifier/blob/main/MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/makemore/master/names.txt'

response = requests.get(url)

with open('names.txt', 'wb') as f:
  f.write(response.content)

In [2]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
# stoi and itos
chars = sorted(list(set(''.join(words))))
stoi = {char:i+1 for i, char in enumerate(chars)}
stoi['.'] = 0
itos = {i:char for char, i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [60]:
# make dataset
block_size = 3 # context length
X, Y = [], []
for w in words:
  #print(w)
  context = [0] * block_size # list of zeros of length block size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '---->', itos[ix])
    context = context[1:] + [ix] # crop to remove previous, append new to end

X = torch.tensor(X)
Y = torch.tensor(Y)

In [61]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [62]:
# C should be a 27 x embedding dimension matrix
C = torch.randn((27, 2))

In [63]:
C[stoi['a']]

tensor([ 0.3448, -0.9542])

In [64]:
aonehot = F.one_hot(torch.tensor(stoi['a']), num_classes=27).float()
aonehot @ C

tensor([ 0.3448, -0.9542])

In [11]:
C[[5, 6, 7]]

tensor([[ 0.4724,  1.8303],
        [-0.6621,  0.3525],
        [ 0.1849, -1.7818]])

In [12]:
C[torch.tensor([0,0,0])]

tensor([[-0.4356, -1.1651],
        [-0.4356, -1.1651],
        [-0.4356, -1.1651]])

In [65]:
for data in X[:5]:
  enc = C[data]
  print(enc)

tensor([[-0.1761, -0.2927],
        [-0.1761, -0.2927],
        [-0.1761, -0.2927]])
tensor([[-0.1761, -0.2927],
        [-0.1761, -0.2927],
        [-0.4398,  0.5333]])
tensor([[-0.1761, -0.2927],
        [-0.4398,  0.5333],
        [-0.5031, -0.0801]])
tensor([[-0.4398,  0.5333],
        [-0.5031, -0.0801],
        [-0.5031, -0.0801]])
tensor([[-0.5031, -0.0801],
        [-0.5031, -0.0801],
        [ 0.3448, -0.9542]])


In [66]:
emb = C[X]
emb.shape

torch.Size([228146, 3, 2])

In [16]:
%time
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.68 µs


torch.Size([32, 6])

In [17]:
%time
torch.flatten(emb, start_dim=1).shape

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.53 µs


torch.Size([32, 6])

In [18]:
%time
torch.cat(torch.unbind(emb, 1), 1).shape

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.96 µs


torch.Size([32, 6])

Idk bruh I think mine is still better

In [19]:
a = torch.arange(18)
a, a.shape

(tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17]),
 torch.Size([18]))

In [20]:
a.view(3, 6) # holy moly where has this been all my life

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11],
        [12, 13, 14, 15, 16, 17]])

In [21]:
%time
emb.view(32, 6).shape

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.01 µs


torch.Size([32, 6])

2 microseconds bruv

In [94]:
C = torch.randn((27, 2))
w1 = torch.randn(6, 100)
b1 = torch.randn(100)
w2 = torch.randn(100, 27)
b2 = torch.randn(27)

wi1 = emb.view(-1, 6) @ w1
wi1b = wi1 + b1
wi1ba = wi1b.tanh()
wi2 = wi1ba @ w2
logits = wi2 + b2
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)

parameters = [C, w1, b1, w2, b2]
for p in parameters:
  p.requires_grad = True
probs.shape

torch.Size([228146, 27])

In [100]:
lre = torch.linspace(-3, 0, 1000)
lrs = 10**lre
lrs

tensor([0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0010, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011, 0.0011,
        0.0011, 0.0011, 0.0011, 0.0012, 0.0012, 0.0012, 0.0012, 0.0012, 0.0012,
        0.0012, 0.0012, 0.0012, 0.0012, 0.0012, 0.0012, 0.0013, 0.0013, 0.0013,
        0.0013, 0.0013, 0.0013, 0.0013, 0.0013, 0.0013, 0.0013, 0.0013, 0.0014,
        0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014, 0.0014,
        0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015, 0.0015,
        0.0015, 0.0016, 0.0016, 0.0016, 0.0016, 0.0016, 0.0016, 0.0016, 0.0016,
        0.0016, 0.0017, 0.0017, 0.0017, 0.0017, 0.0017, 0.0017, 0.0017, 0.0017,
        0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0018, 0.0019,
        0.0019, 0.0019, 0.0019, 0.0019, 0.0019, 0.0019, 0.0019, 0.0020, 0.0020,
        0.0020, 0.0020, 0.0020, 0.0020, 0.0020, 0.0021, 0.0021, 0.0021, 0.0021,
        0.0021, 0.0021, 0.0021, 0.0022, 

In [101]:


for x in range(100):

  # minibatch
  ix = torch.randint(0, X.shape[0], (32, ))


  emb = C[X[ix]]
  wi1 = emb.view(-1, 6) @ w1
  wi1b = wi1 + b1
  wi1ba = wi1b.tanh()
  wi2 = wi1ba @ w2
  logits = wi2 + b2

  print(loss.item())

  for p in parameters:
    p.grad = None

  loss = F.cross_entropy(logits, Y[ix])
  loss.backward()
  for p in parameters:
    p.data -= lrs[x] * 0.05

print(loss.item())

4.854605197906494
3.7189857959747314
5.306065082550049
3.925143241882324
4.923662185668945
3.7951619625091553
4.441082954406738
3.976162910461426
4.871531009674072
3.975682497024536
3.7434303760528564
3.9006505012512207
4.351388454437256
3.238480567932129
4.603549003601074
5.497033596038818
5.13005256652832
3.6410326957702637
4.023245811462402
3.5591306686401367
4.045746326446533
4.239569664001465
3.31107234954834
3.9561963081359863
3.9978864192962646
5.246624946594238
5.785721302032471
4.562099933624268
4.3506669998168945
3.6648221015930176
2.9003663063049316
3.601804494857788
3.5355920791625977
4.006181240081787
3.4920811653137207
4.491122245788574
3.6891348361968994
4.8869500160217285
3.791290044784546
4.626134872436523
3.862677574157715
3.899420738220215
4.3583526611328125
4.123016357421875
3.311497688293457
4.817076206207275
3.721092700958252
4.050938129425049
4.686161518096924
4.565009117126465
3.766775131225586
3.7050487995147705
4.702569961547852
3.0821263790130615
4.3536577224

In [92]:
emb = C[X]
wi1 = emb.view(-1, 6) @ w1
wi1b = wi1 + b1
wi1ba = wi1b.tanh()
wi2 = wi1ba @ w2
logits = wi2 + b2
loss = F.cross_entropy(logits, Y)
loss.item()

2.493295669555664

In [59]:
def prompt(context, max_length=15):
  out = context
  length = 0

  while True:
    length += 1
    emb = C[X]
    wi1 = emb.view(-1, 6) @ w1
    wi1b = wi1 + b1
    wi1ba = wi1b.tanh()
    wi2 = wi1ba @ w2
    logits = wi2 + b2
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    char = itos[torch.multinomial(probs, num_samples=1)]
    if char == '.' or length==max_length:
      break
    out += char
  return out

prompt("br")

KeyError: ignored

In [71]:
torch.randint(0, X.shape[0], (32,))

tensor([ 40252, 193658,  68396,  84080, 131500, 164531, 160830, 156040, 209086,
         65888, 183631,  79638, 178730,  55274,  51907, 130235,   7484,  81981,
         87642,  66139, 107492, 134658, 176115, 126736,   3754,  36199, 103912,
        174984, 194619,  52033, 116802,  89044])